# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
# Load weatherdata.csv
weather_df = pd.DataFrame(pd.read_csv('F:\Matt\GitHome\py-api-challenge\WeatherPy\outputs\weatherdata.csv'))
weather_df = weather_df[['name','country','clouds','humidity','temp','max_temp','wind_spd','lat','lon']]
weather_df['humidity'].max()

100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
# define locations
locations = weather_df[['lat','lon']].astype(float)
humidity = weather_df['humidity'].astype(float)
# configure gmaps
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '1000px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
# add humidity layer
humidity_heatlayer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=True, max_intensity=100,
                                 point_radius = 10)
fig.add_layer(humidity_heatlayer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Remove rows w/ null values
not_null = weather_df[weather_df.notnull()]
# Select for perfect weather
perfect_weather = not_null[(not_null['max_temp']<80) & (not_null['max_temp']>70) & (not_null['wind_spd']<10) & (not_null['clouds']==0)]
perfect_weather

,name,country,clouds,humidity,temp,max_temp,wind_spd,lat,lon
101,Jhinjhāna,IN,0,51,71.24,71.24,2.42,29.516670,77.216667
394,Rohri,PK,0,16,75.78,75.78,7.65,27.692230,68.895058
525,Hickeys Creek,AU,0,60,73.00,73.00,3.00,-30.900000,152.600006
556,Al Ḩayy,IQ,0,19,70.20,70.20,5.03,32.173489,46.043301


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = perfect_weather[['name','country','lat','lon']]
hotel_df.columns = ['City','Country','City lat','City lon']
hotel_df

,City,Country,City lat,City lon
101,Jhinjhāna,IN,29.516670,77.216667
394,Rohri,PK,27.692230,68.895058
525,Hickeys Creek,AU,-30.900000,152.600006
556,Al Ḩayy,IQ,32.173489,46.043301


In [6]:
# Check each city for hotels
hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "-34.551262,-58.537682",
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}
hotel_name = []

for index,row in hotel_df.iterrows():
    params['location'] = {str(row['City lat']) + ',' + str(row['City lon'])}    
    try:
        print(f"trying {row['City']}")
        response = requests.get(hotel_url, params=params).json()
        results = response['results']
        print(f"{results[0]['name']}")
        hotel_name.append(results[0]['name'])
    except:
        print('No hotels found')
        hotel_name.append('')
hotel_df['Hotel Name'] = hotel_name
hotel_df

trying Jhinjhāna
Ajanta saiting
trying Rohri
AD Kukreja House
trying Hickeys Creek
No hotels found
trying Al Ḩayy
Neighborhood youth center


C:\Users\Ward33\anaconda3\envs\PythonData2\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Country,City lat,City lon,Hotel Name
101,Jhinjhāna,IN,29.516670,77.216667,Ajanta saiting
394,Rohri,PK,27.692230,68.895058,AD Kukreja House
525,Hickeys Creek,AU,-30.900000,152.600006,
556,Al Ḩayy,IQ,32.173489,46.043301,Neighborhood youth center


In [7]:
# align column names with provided code below
hotel_df.rename(columns = {'City lat':'Lat','City lon':'Lng'}, inplace=True)
hotel_df

C:\Users\Ward33\anaconda3\envs\PythonData2\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,City,Country,Lat,Lng,Hotel Name
101,Jhinjhāna,IN,29.516670,77.216667,Ajanta saiting
394,Rohri,PK,27.692230,68.895058,AD Kukreja House
525,Hickeys Creek,AU,-30.900000,152.600006,
556,Al Ḩayy,IQ,32.173489,46.043301,Neighborhood youth center


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…